In [1]:
print ('Hello')

Hello


In [42]:
from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            if ('news_df') in filename and ('16') in filename:
                filepath = os.path.join(root, filename) 
                file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def main(): 
    # path to folder which needs to be zipped 
    directory = './'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('2016.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')
main()    

Following files will be zipped:
./2016-12_news_df.pkl
./2016-11_news_df.pkl
All files zipped successfully!


In [2]:
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git
import pandas #For DataFrames
import numpy as np #For arrays
import pickle #if you want to save layouts
import os
import re
import zipfile
import sys
import spacy
import time
nlp=spacy.load('en_core_web_sm')

In [21]:
#final
def loadDavies(address, fname, corpus_style="text", num_files=10000, return_raw=True):
    texts_raw = {}
    article_count=0
    file=address+'/'+fname
    if corpus_style in file:
                print(type(file),file)
                zfile = zipfile.ZipFile(file)
                for file in zfile.namelist():
                    #check for us source
                    if file.lower().find('us')==-1:
                        continue
                    texts_raw[file] = []
                    with zfile.open(file) as f:
                        for article in f:
                            article_lower = article.decode().lower()
                            #only take articles with election in them
                            finde = 'election' in article_lower
                            if (finde):
                                article_count+=1
                                texts_raw[file].append(article)
    print (article_count)                            
    if return_raw:
        return(texts_raw)

In [40]:
corpus_name = "/project2/soci40133/Davies_corpora/NOW"
flist=os.listdir(corpus_name + "/")
s_dict={}
for f in flist:
    #if ('text-18' in f or 'text-16-10' in f or 'text-16-11' in f or 'text-17' in f )and 'lexicon' not in f and'wlp' not in f and 'db' not in f and not 'text-18-11' in f and not 'text-18-12' in f:
    if ('text-16-11' in f or 'text-16-12' in f):
        s_dict[f]=f.replace("text","sources")

print(list(s_dict.keys()))
print(len(s_dict.keys()))


['text-16-12.zip', 'text-16-11.zip']
2


In [20]:
#TODO:get rid of everything before the first p tag and store headline seperately
def clean_raw_text(raw_articles):
    clean_articles = []
    for text in raw_articles:
        try:
            text = text.decode("utf-8")
            clean_article = text.replace(" \'m", "'m").replace(" \'ll", "'ll").replace(" \'re", "'re").replace(" \'s", "'s").replace(" \'re", "'re").replace(" n\'t", "n't").replace(" \'ve", "'ve").replace(" /'d", "'d")\
                            .replace("<h>","").replace("<p>","").replace("@@","")
            clean_articles.append(clean_article)
        except AttributeError:
            # print("ERROR CLEANING")
            # print(text)
            continue
        except UnicodeDecodeError:
            # print("Unicode Error, Skip")
            continue
    return clean_articles

In [19]:
def word_tokenize(text, remove_stop_words=False):
    tokenized = []
    customize_stop_words = ['is']
    # pass word list through language model.
    doc = nlp(text)
    
    for w in customize_stop_words:
        nlp.vocab[w].is_stop = True
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            if remove_stop_words:
                if not token.is_stop: 
                   tokenized.append(token.text)
                else:
                    continue
            else:
                tokenized.append(token.text)
    
    return tokenized

In [12]:
#word_tokenize('Mr. aruna is bad', False)

In [34]:
def Process(news_raw,file_name,source_name):
    t0 = time.time()
    ta = time.time()
    a_count =0
    news_texts = {}
    just_texts = {}
    for file in news_raw:
        print('File_name',file)
        #article=news_raw[file]
        articles = clean_raw_text(news_raw[file])
        article_cleaned=articles[0]
        for article in articles:
            if (a_count%100)==0:
                t=time.time()
                print(t-ta)
                ta=time.time()
                print('Processed',a_count)

            ##where we sentence break, tokenize sentences, flatten, etc.
            tokenized_sents = [lucem_illud_2020.word_tokenize(s) for s in lucem_illud_2020.sent_tokenize(article)]
            normalized_sents = [lucem_illud_2020.normalizeTokens(s) for s in tokenized_sents]
            #remove empty lists
            normalized_sents =  list(filter(None, normalized_sents))
            #print(normalized_sents)
            tokens = [item for sublist in normalized_sents for item in sublist]
            tid=article.split(" ")[0]
            #print(tid)
            #print(tokens)
            #tokens_spacy= word_tokenize(article)
            try:
                if tid in list(news_texts.values()):
                    print (tid, 'already seen...bad coding')
                news_texts[tid] = (tokens,normalized_sents)
                just_texts[tid] = article
                a_count+=1
            except IndexError:
                print("Missing: ",txts[0][2:])
                continue
    t1 = time.time()
    print ("Total time", t1-t0)   
#     try:
#         assert(len(list(news_texts.keys()))==a_count)
#     except:
#         print(len(list(news_texts.keys())))
#         print(a_count)
    ids=set((news_texts.keys()))
    sources = []
    zfile = zipfile.ZipFile(corpus_name + "/"+source_name)
    for file in zfile.namelist():
        with zfile.open(file) as f:
            for line in f:
                sources.append(line)
    sl=[]
    ecount=0
    for s in sources:
        try:
            sl.append(s.decode().split("\t")[0].strip())
            #print(s.decode().split("\t")[2].strip())
        except UnicodeDecodeError:
            ecount+=1
            continue
    print("Total source documents", len(sources))
    print("Files with decode errors:", ecount)
    print("Files whose sources intersect with the files we have collected", len(set(sl).intersection(ids)))
    print ("Total files", len(ids))
    news_df = pandas.DataFrame(columns=["title", "date", "country", "source", "url", "text", "tokens","normalized_tokens"])
    count=0
    sl = []
    for news in sources:
        try:
            tid, total_words, date, country, source, url, title = news.decode("utf-8").split("\t")
        except UnicodeDecodeError:
            continue
        try:
            news_df.loc[tid.strip()] = [title.strip(), date.strip(), country.strip(), source.strip(), url.strip(), just_texts[tid.strip()], news_texts[tid.strip()][0],news_texts[tid.strip()][1]]
            sl.append(tid)
            count+=1
        except KeyError:
            continue
    #the tids not found in sources
    for tids in list(ids.difference(set(sl))):
        contents = ([""]+[file_name.replace('.zip','').replace('text-','')+'-00'])+['US']+ ([""]*2)+ [just_texts[tids], news_texts[tids][0],news_texts[tids][1]]
        #print ((contents))
        #print (type(contents))
        #try:
        news_df.loc[tids.strip()] = contents
        #except:
            #print (tids)
    print('Shape of data_frame', news_df.shape)
    print('Size of keys', len(ids))
    news_df.to_pickle(file_name.replace('text-','20').replace('.zip','')+'_news_df.pkl')
    print('File saved at',file_name.replace('text-','20').replace('.zip','')+'_news_df.pkl')

In [41]:
corpus_name = "/project2/soci40133/Davies_corpora/NOW"
df_count = 0
print (len(s_dict.keys()))
for f in list(s_dict.keys()):
    file_name=f
    source_name=s_dict[file_name]
    news_raw=loadDavies(corpus_name, file_name)
    Process(news_raw,file_name,source_name)
    df_count+=1
    print('Processed ',df_count,'of ',len(s_dict.keys()))


2
<class 'str'> /project2/soci40133/Davies_corpora/NOW/text-16-12.zip
3092
File_name text_16-12-US.txt
0.2551898956298828
Processed 0
14.6622314453125
Processed 100
11.963776111602783
Processed 200
12.512112379074097
Processed 300
13.05117917060852
Processed 400
12.340938091278076
Processed 500
12.661717414855957
Processed 600
11.571814060211182
Processed 700
14.099770545959473
Processed 800
12.132935523986816
Processed 900
13.307594776153564
Processed 1000
11.71561574935913
Processed 1100
12.390451669692993
Processed 1200
11.071134805679321
Processed 1300
17.184241771697998
Processed 1400
15.59457802772522
Processed 1500
13.074754238128662
Processed 1600
15.33001160621643
Processed 1700
14.212183952331543
Processed 1800
13.374427795410156
Processed 1900
13.36827826499939
Processed 2000
12.171555995941162
Processed 2100
14.62836766242981
Processed 2200
17.68419909477234
Processed 2300
13.861583948135376
Processed 2400
15.863219976425171
Processed 2500
14.574506998062134
Processed 2600


In [ ]:
print(article_cleaned)

In [ ]:
ids=set((news_texts.keys()))

In [ ]:
len(ids)

In [ ]:
# source matching
sources = []
zfile = zipfile.ZipFile(corpus_name + "/"+source_name)

for file in zfile.namelist():
    with zfile.open(file) as f:
        for line in f:
            sources.append(line)

In [ ]:
sl=[]
ecount=0
for s in sources:
    try:
        sl.append(s.decode().split("\t")[0].strip())
        #print(s.decode().split("\t")[2].strip())
    except UnicodeDecodeError:
        ecount+=1
        continue
    

In [ ]:
print("Total source documents", len(sources))
print("Files with decode errors:", ecount)
print("Files whose sources intersect with the files we have collected", len(set(sl).intersection(ids)))
print ("Total files", len(ids))

In [ ]:
#news_texts[list(set(sl).intersection(ids))[0]]

In [ ]:
news_df = pandas.DataFrame(columns=["title", "date", "country", "source", "url", "text", "tokens","normalized_tokens"])
count=0
sl = []
for news in sources:
    try:
        tid, total_words, date, country, source, url, title = news.decode("utf-8").split("\t")
    except UnicodeDecodeError:
        continue
    try:
        news_df.loc[tid.strip()] = [title.strip(), date.strip(), country.strip(), source.strip(), url.strip(), just_texts[tid.strip()], news_texts[tid.strip()][0],news_texts[tid.strip()][1]]
        sl.append(tid)
        count+=1
    except KeyError:
        continue
#the tids not found in sources
for tids in list(ids.difference(set(sl))):
    contents = ([""]*2)+['US']+ ([""]*2)+ [just_texts[tids], news_texts[tids][0],news_texts[tids][1]]
    #print ((contents))
    #print (type(contents))
    #try:
    news_df.loc[tids.strip()] = contents
    #except:
        #print (tids)

In [ ]:
news_df.shape

In [ ]:
news_df.head()

In [ ]:
#sanity check
#news_df.loc['15159083']['tokens'][:100]

In [ ]:
#news_df.to_csv('2016-11_news_df.csv')
news_df.to_csv(file_name.replace('text-','20').replace('.zip','')+'_news_df.csv')